# WEB SCRAPING ASSIGNMENT - 3

In [49]:
#import all libraries

from bs4 import BeautifulSoup
import requests
import selenium
import pandas as pd
from selenium import webdriver
import time
import re
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None, "display.max_columns", None)
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Q 1

Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
#Connecting to the web driver
driver=webdriver.Chrome(r"C:\Users\Acer\Desktop\Internship 11 - Project 1 - Avinash Carneiro\chromedriver.exe")

#Specifying the url of the webpage
driver.get("https://www.amazon.in/")

#Assigning the "Search_Keys" option to search the Specific Keyword on Amazon
Search_Keys = input("Type the Keyword to Search on Amazon:----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Amazon.in
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@id='twotabsearchtextbox']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@id='nav-search-submit-text']"))).click()


Type the Keyword to Search on Amazon:----> Guitar


# Q 2

In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@id='nav-search-submit-text']"))).click()

#Getting the "href" value/URL of the product in first 3 pages
URT1=[]
for i in range(0, 3):
    for i in driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']"):
        URT1.append(i.get_attribute('href'))

    driver.find_element_by_xpath("//li[@class='a-last']").click()
    time.sleep(3)
        
#Lets create required empty lists required for the 
Avg_Ratings=[]
Brand=[]
No_of_Ratings=[]
Price=[]
Return_Exchange=[]
Expected_Delivery=[]
Availability=[]
Other_Details=[]
Product_URL=[]

#loop for all the required data of product
for url in URT1:
    driver.get(url)
    
#To find the Avg_Ratings of the product
    try:                  
        rate=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']")
        rate.click()                                                      
        rating=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']")
        Avg_Ratings.append(rating.text)
        
    except NoSuchElementException   as e:
        Avg_Ratings.append("-")
               
#To find the Brand Name & Details of the product
    try: 
        brand=driver.find_element_by_xpath("//span[@id='productTitle']")
        Brand.append(brand.text)
        
    except NoSuchElementException   as e:
        Brand.append("-")       
        
#To find the Brand Name & Details of the product  
    try: 
        ratings=driver.find_element_by_xpath("//span[@class='a-size-base a-color-secondary']")
        No_of_Ratings.append(ratings.text)
        
    except NoSuchElementException   as e:
        No_of_Ratings.append("-")
        
        
#To find the Price of the product 
    try: 
        price=driver.find_element_by_xpath("//td[@class='a-span12']")
        Price.append(price.text)
        
    except NoSuchElementException   as e:
        Price.append("-")
              
#To find the Return & Exchange policy of the product 
    try: 
        
        ret_exchange=driver.find_element_by_xpath("//div[@class='a-column a-span3 a-text-center icon-container  icon-grid-width']")#locating the brand
        Return_Exchange.append(ret_exchange.text)
        
    except NoSuchElementException   as e:
        Return_Exchange.append("-")
              
#To find the Expected Delivery policy of the product 
    try: 
        
        Delivery=driver.find_element_by_xpath("//div[@id='dynamicDeliveryMessage']")
        Expected_Delivery.append(Delivery.text)
        
    except NoSuchElementException   as e:
        Expected_Delivery.append("-")        
        
#To find the Availability of the product 
    try: 
        
        availability=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")
        Availability.append(availability.text)
        
    except NoSuchElementException   as e:
        Availability.append("-")       
        
#To find the Other Details of the product
    try: 
        
        Details=driver.find_element_by_xpath("//div[@id='feature-bullets']")
        Other_Details.append(Details.text)
        
    except NoSuchElementException   as e:
        Other_Details.append("-")

#Creating the dataframe for the Searched Keyword/Product searched on Amazon

DF=pd.DataFrame({})
DF['Brand']=Brand
DF['No_of_Ratings']=No_of_Ratings
DF['Avg_Ratings']=Avg_Ratings
DF['Price']=Price
DF['Expected_Delivery']=Expected_Delivery
DF['Other Details']=Other_Details


#Creating the dataframe for Product_Vertical
Product_Vertical=pd.DataFrame({})

#Brand Name of the Guitar
Product_Vertical['Brand Name'] = DF['Brand'].str.split(' ').str[0]

#Name Of Product/Guitar mentioned on Amazon
Product_Vertical['Name Of Product'] = DF['Brand'].str.split(' ').str[1:]
Product_Vertical['Name Of Product'] = DF['Brand'].str.replace('[.,(),]', '')

#Average Ratings for the Guitar 
Product_Vertical['Average Rating']=DF['Avg_Ratings'].str.split(' ').str[0]

#No_of_Ratings for the Guitar 
Product_Vertical['No Of Ratings'] = DF['No_of_Ratings'].str.split(' ').str[0]

#Price of the Guitar
Product_Vertical['Price_INR'] = DF['Price'].str.split(' ').str[1:2]

#Return policy of product
Product_Vertical['Return Policy'] = Return_Exchange

#Expected Delivery of product
Product_Vertical['Expected Delivery'] = DF['Expected_Delivery'].str.split(' ').str[2:5].replace('[.,(),]', '')

#Availability of product
Product_Vertical['Product Availability'] = Availability

#Other Details of product
Product_Vertical['Other Details'] = DF['Other Details'].str.replace('\n', ' ')

#URL of product
Product_Vertical['Product URL'] = URT1

print('The DataFrame for Searched product on https://www.amazon.in/ is :')

print("\n")
Product_Vertical

The DataFrame for Searched product on https://www.amazon.in/ is :




,Brand Name,Name Of Product,Average Rating,No Of Ratings,Price_INR,Return Policy,Expected Delivery,Product Availability,Other Details,Product URL
0,Kadence,Kadence Frontier Series Natural Acoustic Guita...,3.8,52,"[5,687.00]",10 Days Replacement,"[Sunday,, Feb, 7]",In stock.,"Product Type : Acoustic Guitar Brand:Kadence,F...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Kadence Frontier Series Acoustic Guitar With/W...,3.9,404,"[4,499.00]",10 Days Replacement,"[Sunday,, Feb, 7]",In stock.,Scale Size(In Inchces):39 Fretboard Material:R...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Medellin,Medellin MED-BLU-C Linden Wood Acoustic Guitar,3.8,248,"[2,399.00]",10 Days Replacement,"[Sunday,, Feb, 7]",In stock.,Material: Wood Colour: Blue Acoustic Guitar › ...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Grail,Grail Allure A190CE SAS Semi Acoustic Guitar A...,4.8,9,"[7,429.00]",10 Days Replacement,"[Tuesday,, Feb, 9]",In stock.,"39"" Cutaway Electro Acoustic guitar 3 Band Equ...",https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,Juârez Acoustic Guitar 38 Inch Cutaway 038C wi...,3.9,"7,197","[2,499.00]",10 Days Replacement,"[Sunday,, Feb, 7]",In stock.,"About this item Black Glossy Finish, Number of...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
5,Intern,Intern INT-38C Acoustic Guitar Kit With Bag St...,3.9,"4,149","[2,490.00]",10 Days Replacement,"[Sunday,, Feb, 7]",In stock.,About this item Great looks with an innovative...,https://www.amazon.in/Intern-INT-38C-Acoustic-...
6,Juarez,Juarez JRZ38C Acoustic Guitar 38 Inch Cutaway ...,4,"2,495","[2,499.00]",10 Days Replacement,"[Monday,, Feb, 8]",In stock.,"About this item Black glossy finish, number of...",https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
7,AmazonBasics,AmazonBasics Guitar Picks Mixed Colors Cellulo...,4.6,391,[299.00],10 Days Replacement,"[Monday,, Feb, 8]",In stock.,10-piece set of flat guitar picks includes the...,https://www.amazon.in/AmazonBasics-Guitar-Colo...
8,AmazonBasics,AmazonBasics Adjustable Guitar Strap For Elect...,4.5,157,[449.00],10 Days Replacement,"[Sunday,, Feb, 7]",In stock.,"Adjustable guitar strap for an electric, acous...",https://www.amazon.in/AmazonBasics-Adjustable-...
9,AmazonBasics,AmazonBasics Adjustable Guitar Strap for Elect...,4.6,435,[399.00],10 Days Replacement,"[Sunday,, Feb, 7]",In stock.,"Adjustable guitar strap for an electric, acous...",https://www.amazon.in/AmazonBasics-Adjustable-...


# Q 3

Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

1.To check the first "100 fruits" images on https://images.google.com/

In [2]:
#To check the first 100 fruits on https://images.google.com/

#Connecting to the web driver
driver=webdriver.Chrome(r"C:\Users\Acer\Desktop\Internship 11 - Project 1 - Avinash Carneiro\chromedriver.exe")
#Specifying the url of the webpage
driver.get("https://images.google.com/")


#1. To check the first 100 fruits on https://images.google.com/
print('1. To check the first "100 fruits" on https://images.google.com/')  
print("\n")

#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()


#Getting the "src" image value of the word on Google
URT1=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
        URT1.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT1.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT1.append(i.get_attribute('alt'))
    

#Checking the length URT1
print('> Lenght of Searched Keyword :', len(URT1))

#Dataframe for Searched keyword
Searched_Keyword=pd.DataFrame({})
Searched_Keyword['Images']=URT1
Searched_Keyword.head()
print("\n")

#Checking for the null value in Searched_Keyword
print('Checking for the null values in Searched_Keyword')
print('> Any null value present :', Searched_Keyword.isnull().values.any())
print('> Sum of all null values :', Searched_Keyword.isnull().sum())
print("\n")
print('Info of all Searched_Keyword')
Searched_Keyword.info()
print("\n")

#Dataframe for Fruit_Images
Fruit_Images=pd.DataFrame({})
Fruit_Images['Fruit Images']=Searched_Keyword['Images']

#Checking the Shape of Dataframe Fruit_Images
print('> Shape of dataframe Fruit_Images :', Fruit_Images.shape)
print("\n")

#Selecting the 100 images of the searched keyword - fruits
print('The 100 images of the searched keyword - "fruits" on https://images.google.com/ is :')
Fruit_Image=Fruit_Images[0:100]
print('> Shape of Dataframe :', Fruit_Image.shape)
Fruit_Image.head(20)

1. To check the first "100 fruits" on https://images.google.com/


> Type the Keyword to Search on https://images.google.com/  :----> fruits
> Lenght of Searched Keyword : 147


Checking for the null values in Searched_Keyword
> Any null value present : True
> Sum of all null values : Images    49
dtype: int64


Info of all Searched_Keyword
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Images  98 non-null     object
dtypes: object(1)
memory usage: 1.3+ KB


> Shape of dataframe Fruit_Images : (147, 1)


The 100 images of the searched keyword - "fruits" on https://images.google.com/ is :
> Shape of Dataframe : (100, 1)


,Fruit Images
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


2. To check the first "100 cars" images on https://images.google.com/

In [3]:
#To check the first 100 cars on https://images.google.com/

#Connecting to the web driver
driver=webdriver.Chrome(r"C:\Users\Acer\Desktop\Internship 11 - Project 1 - Avinash Carneiro\chromedriver.exe")
#Specifying the url of the webpage
driver.get("https://images.google.com/")


#1. To check the first 100 fruits on https://images.google.com/
print('2. To check the first "100 cars" on https://images.google.com/')  
print("\n")

#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()


#Getting the "src" image value of the word on Google
URT2=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT2.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT2.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT2.append(i.get_attribute('alt'))
    

#Checking the length URT2
print('> Lenght of Searched Keyword :', len(URT2))

#Dataframe for Searched keyword
Searched_Keyword=pd.DataFrame({})
Searched_Keyword['Images']=URT2
Searched_Keyword.head()
print("\n")

#Checking for the null value in Searched_Keyword
print('Checking for the null values in Searched_Keyword')
print('> Any null value present :', Searched_Keyword.isnull().values.any())
print('> Sum of all null values :', Searched_Keyword.isnull().sum())
print("\n")
print('Info of all Searched_Keyword')
Searched_Keyword.info()
print("\n")

#Dataframe for Fruit_Images
Cars_Images=pd.DataFrame({})
Cars_Images['Car Images']=Searched_Keyword['Images']

#Checking the Shape of Dataframe Cars_Images
print('> Shape of dataframe Cars_Images :', Cars_Images.shape)
print("\n")

#Selecting the 100 images of the searched keyword - fruits
print('The 100 images of the searched keyword - "cars" on https://images.google.com/ is :')
Car_Images=Cars_Images[0:100]
print('> Shape of Dataframe :', Car_Images.shape)
Car_Images.head(20)

2. To check the first "100 cars" on https://images.google.com/


> Type the Keyword to Search on https://images.google.com/  :----> cars
> Lenght of Searched Keyword : 147


Checking for the null values in Searched_Keyword
> Any null value present : True
> Sum of all null values : Images    49
dtype: int64


Info of all Searched_Keyword
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Images  98 non-null     object
dtypes: object(1)
memory usage: 1.3+ KB


> Shape of dataframe Cars_Images : (147, 1)


The 100 images of the searched keyword - "cars" on https://images.google.com/ is :
> Shape of Dataframe : (100, 1)


,Car Images
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


3. To check the first "100 machine Learning" images on https://images.google.com/

In [4]:
#To check the first 100 fruits on https://images.google.com/

#Connecting to the web driver
driver=webdriver.Chrome(r"C:\Users\Acer\Desktop\Internship 11 - Project 1 - Avinash Carneiro\chromedriver.exe")
#Specifying the url of the webpage
driver.get("https://images.google.com/")


#1. To check the first 100 fruits on https://images.google.com/
print('1. To check the first "100 fruits" on https://images.google.com/')  
print("\n")

#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()


#Getting the "src" image value of the word on Google
URT1=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
        URT1.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT1.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT1.append(i.get_attribute('alt'))
    

#Checking the length URT1
print('> Lenght of Searched Keyword :', len(URT1))

#Dataframe for Searched keyword
Searched_Keyword=pd.DataFrame({})
Searched_Keyword['Images']=URT1
Searched_Keyword.head()
print("\n")

#Checking for the null value in Searched_Keyword
print('Checking for the null values in Searched_Keyword')
print('> Any null value present :', Searched_Keyword.isnull().values.any())
print('> Sum of all null values :', Searched_Keyword.isnull().sum())
print("\n")
print('Info of all Searched_Keyword')
Searched_Keyword.info()
print("\n")

#Dataframe for "Machine Learning"
ML=pd.DataFrame({})
ML['Machine Learning']=Searched_Keyword['Images']

#Checking the Shape of Dataframe ML
print('> Shape of dataframe ML :', ML.shape)
print("\n")

#Selecting the 100 images of the searched keyword - Machine Learning
print('The 100 images of the searched keyword - "fruits" on https://images.google.com/ is :')
Machine_Learning=ML[0:100]
print('> Shape of Dataframe :', Machine_Learning.shape)
Machine_Learning.head(20)

1. To check the first "100 fruits" on https://images.google.com/


> Type the Keyword to Search on https://images.google.com/  :----> Machine Learning
> Lenght of Searched Keyword : 147


Checking for the null values in Searched_Keyword
> Any null value present : True
> Sum of all null values : Images    49
dtype: int64


Info of all Searched_Keyword
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Images  98 non-null     object
dtypes: object(1)
memory usage: 1.3+ KB


> Shape of dataframe ML : (147, 1)


The 100 images of the searched keyword - "fruits" on https://images.google.com/ is :
> Shape of Dataframe : (100, 1)


,Machine Learning
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


# Q4

Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [60]:
#Connecting to the web driver
driver=webdriver.Chrome(r"C:\Users\Acer\Desktop\Internship 11 - Project 1 - Avinash Carneiro\chromedriver.exe")
#Specifying the url of the webpage
driver.get("https://www.flipkart.com/")

# Asking for user input
prod = input("Type the of the mobile phone you want to search on https://www.flipkart.com/ : ")
time.sleep(3)

#Button to close login popup
try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')  
    login_X_button.click() 
except NoSuchElementException : 
    print("No Login page")
    
#Finding the search bar 
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')   
search_bar.clear()               
search_bar.send_keys(prod) 

# Inputing keyword to search
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    
search_button.click()

Type the of the mobile phone you want to search on https://www.flipkart.com/ : Samsung Galaxy M51


0

In [62]:
Brand1=[]
Price1=[]

#Getting the "href" value/URL of the product on first page
URL1 = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    URL1.append(url.get_attribute("href"))
    
len(URL1)

#loop for all the required data of product
for url in URL1:
    driver.get(url)
    
#To find the Brand Name & Details of the product
    try: 
        brand=driver.find_element_by_xpath("//span[@class='B_NuCI']")
        Brand1.append(brand.text)
    except NoSuchElementException   as e:
        Brand1.append("-")
        
#To find the Price of the product
    try: 
        price=driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
        Price1.append(price.text)
    except NoSuchElementException   as e:
        Price1.append("-") 


In [65]:
len(Brand1), len(Price1)

(24, 24)

In [ ]:
#INCOMPLETE

# Q5

Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [37]:
#To check the geospatial coordinates on https://www.google.com/maps/

#Connecting to the web driver
driver=webdriver.Chrome(r"C:\Users\Acer\Desktop\Internship 11 - Project 1 - Avinash Carneiro\chromedriver.exe")

#Specifying the url of the webpage
driver.get("https://www.google.com/maps/")

#To check the geospatial coordinates on https://www.google.com/maps/
print('To check the "Latitude & Longitude" of the City/Town/Place on https://www.google.com/maps')  
print("\n")

#Assigning the "Search_Keys" option to search the Specific Keyword on Google Maps
Search_Keys = input("> Type the 'City/Town/Place' name to Search on https://www.google.com/maps  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google Maps
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@autofocus='autofocus']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//button[@id='searchbox-searchbutton']"))).click()
print("\n")

To check the "Latitude & Longitude" of the City/Town/Place on https://www.google.com/maps


> Type the 'City/Town/Place' name to Search on https://www.google.com/maps  :----> Mangalore




In [42]:
#To find the Latitude & Longitude of searched City
try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    print("\n")
    
#To split the coordinates of the searhed place
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.com/maps/place/Mangalore,+Karnataka/@12.9229829,74.7820227,12z/data=!3m1!4b1!4m5!3m4!1s0x3ba35a4c37bf488f:0x827bbc7a74fcfe64!8m2!3d12.9141417!4d74.8559568


Latitude = 12.9229829, Longitude = 74.7820227


# Q 6

Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [47]:
#REFERENCE - Assignment Solution 3

#Connecting to the web driver
driver=webdriver.Chrome(r"C:\Users\Acer\Desktop\Internship 11 - Project 1 - Avinash Carneiro\chromedriver.exe")

#Specifying the url of the webpage
driver.get("https://trak.in/")

#Selecting the "Funding Deals" link in the website
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

#Url of the trak.in website
url_string = driver.current_url
print("The Current URL: ", url_string)

#Dataframe
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


#Scraping the data from website
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

The Current URL:  https://trak.in/india-startup-funding-investment-2015/


,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"
